<a href="https://colab.research.google.com/github/lehuong240823/rag-company-knowledge-consultant-chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Started

In [2]:
# @title Install Libraries
!pip -q install pinecone
!pip -q install langchain_community
!pip -q install langchain_openai
!pip -q install langchain_pinecone
!pip -q install langchain-huggingface
!pip -q install langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.1 MB/s eta 0:00:00


In [3]:
# @title Import Libraries
import os
from google.colab import userdata
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [4]:
# @title Set Environment Variables
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_INDEX_NAME'] = 'langchain-test-index'
os.environ['OPENAI_API_KEY'] = userdata.get('OPENROUTER_API_KEY')
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['PINECONE_INDEX_NAME'] = userdata.get('PINECONE_INDEX_NAME')
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')

# Method

In [5]:
# @title Get Pinecone Index
def get_index(index_name=os.environ['PINECONE_INDEX_NAME']):
  pc = Pinecone()
  if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

  return pc.Index(index_name)

In [6]:
# @title Text Splitter
def init_text_splitter(chunk_size=1000, chunk_overlap=0):
  return CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [7]:
# @title Add Documents to Vector Store
def add_documents(path):
  loader = TextLoader(path)
  documents = loader.load()
  print(f"Loaded {len(documents)} document(s) from {path}")
  docs = text_splitter.split_documents(documents)
  vectorstore.add_documents(docs)
  print(f"Added {len(docs)} text chunk(s) to vector store.")

In [8]:
# @title Add Texts to Vector Store
def add_texts(text):
  texts = text_splitter.split_text(text)
  vectorstore.add_texts(texts)
  print(f"Added {len(texts)} text chunks to vector store.")

In [9]:
# @title Initialize Components

index = get_index()

llm = ChatOpenAI(model='qwen/qwen3-8b:free')

embeddings = HuggingFaceEndpointEmbeddings(model='Qwen/Qwen3-Embedding-8B')

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

text_splitter = init_text_splitter()

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# Evaluator

In [ ]:
query = "công ty này ở phường nào"
response = qa_chain.invoke({"query": query})

print(response)

{'query': 'công ty này ở phường nào', 'result': 'Tôi không biết công ty cụ thể bạn đang hỏi là công ty nào. Bạn có thể cho biết tên công ty hoặc thêm thông tin liên quan đến địa chỉ của nó được không? Tôi sẽ giúp bạn tìm kiếm thông tin chính xác hơn.', 'source_documents': []}


In [15]:
import json
dataset_path = userdata.get('DATASET_PATH')

In [16]:
from langsmith import Client

client = Client()

examples = json.load(open(dataset_path))

dataset_name = "FQ&A"
dataset = client.create_dataset(dataset_name='FQA')
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)


{'example_ids': ['e81bee28-69f5-4c43-8c95-7b2f1474e93b',
  'd012d491-07a0-4cfb-9ac8-d95aeb785176',
  'bf8463d8-79f1-401a-bc8b-716bda9ceaf8'],
 'count': 3}